## private submit

* 당진태양광 : dangjin
* 당진수상태양광 : dangjin_floating
* 당진자재창고태양광 : dangjin_warehouse
* 울산태양광 : ulsan

기상청 데이터 경로 및 파일 이름 : /new_data
* 당진 데이터 dangjin_fcst_h2320.csv
* 울산 데이터 ulsan_fcst_h2320.csv
* /data/sample_submission.csv

#### <span style="color:red"><b>해당 코드는 API 사용이 포함되어 있습니다.</b> </span> 
따라서 'API 키 값을 넣어주세요' 라는 문구를 포함하였습니다. <br>
dacon 측의 요청대로 API 키 값을 가린 부분입니다. <br>
코드 사용 시 반드시 API 키 값 첨부 후 사용 바랍니다.

In [ ]:
import urllib.request as ul
import xmltodict
import json
import sys
import io

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from urllib.request import urlopen
from urllib.request import Request 
from urllib.parse import urlencode, quote_plus
from datetime import date, timedelta
from tensorflow.keras.models import load_model

#### <span style="color:blue">* private 평가를 위해 날짜를 설정하는 부분입니다. </span>
따라서 이 코드를 사용해서 리더보드 복원 시에 dacon 측에서는 날짜를 변경하셔야 합니다.

In [ ]:
today = date.today().strftime("%Y%m%d")

## 당진 내일 기상 예보 받아오기(충남 당진시 석문면 기준, x=53, y=114)

In [ ]:
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst?serviceKey=API 키 값을 넣어주세요&numOfRows=113&base_date='+str(today)+'&base_time=2000&nx=53&ny=114'

requestd = Request(url)
requestd.get_method = lambda: 'GET'
response_body = urlopen(requestd).read()
resp = xmltodict.parse(response_body)
response20 = json.loads(json.dumps(resp))

##### <span style="color:blue">* private 평가를 위해 날짜를 설정하는 부분입니다. </span>
date = date.today() + timedelta(1) <br>
따라서 이 코드를 사용해서 리더보드 복원 시에 dacon 측에서는 날짜를 변경하셔야 합니다.

In [ ]:
dangjin_fcst_df = pd.DataFrame()

date = date.today() + timedelta(1)
dangjin_fcst_df['Forecast_time'] = [f'{date} {hour}:00' for hour in range(24)]
row_idx = 0 #20시 예보 데이터를 삽입하기 위한 초기 위치 변수 입니다.

for i, data in enumerate(response20['response']['body']['items']['item']):
    if data['category']=='REH':
        dangjin_fcst_df.loc[row_idx, 'Humidity'] = float(data['fcstValue'])
        print('category:Humidity,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
    elif data['category']=='T3H':
        dangjin_fcst_df.loc[row_idx, 'Temperature'] = float(data['fcstValue'])
        print('category:Temperature,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
    elif data['category']=='SKY':
        dangjin_fcst_df.loc[row_idx, 'Cloud'] = float(data['fcstValue'])
        print('category:Cloud,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
    elif data['category']=='PTY':
        dangjin_fcst_df.loc[row_idx, 'PrecipitationForm'] = float(data['fcstValue'])
        print('category:PrecipitationForm,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])            
    elif data['category']=='POP':
        dangjin_fcst_df.loc[row_idx, 'PrecipitationProb'] = float(data['fcstValue'])
        print('category:PrecipitationProb,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])                       
    elif data['category']=='R06':
        dangjin_fcst_df.loc[row_idx, 'Precipitation'] = float(data['fcstValue'])
        print('category:Precipitation,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])                        
    elif data['category']=='S06':
        dangjin_fcst_df.loc[row_idx, 'Snow'] = float(data['fcstValue'])
        print('category:Snow,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])                                              
    elif data['category']=='VEC':
        dangjin_fcst_df.loc[row_idx, 'WindDirection'] = float(data['fcstValue'])
        print('category:WindDirection,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
    elif data['category']=='WSD':
        dangjin_fcst_df.loc[row_idx, 'WindSpeed'] = float(data['fcstValue'])
        print('category:WindSpeed,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'], '\n')
        row_idx+=3

3시간 간격으로 20시의 예보 데이터를 추출합니다.

In [ ]:
today = date.today().strftime("%Y%m%d")

url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst?serviceKey=API 키 값을 넣어주세요&numOfRows=113&base_date='+str(today)+'&base_time=2300&nx=53&ny=114'

requestd = Request(url)
requestd.get_method = lambda: 'GET'
response_body = urlopen(requestd).read()
resp = xmltodict.parse(response_body)
response23 = json.loads(json.dumps(resp))

In [ ]:
row_idx = 3 #00시를 제외한 23시 예보 데이터를 덮어쓰기하기 위한 초기위치 변수 입니다.

for i, data in enumerate(response23['response']['body']['items']['item']):
    if data['category']=='REH':
        dangjin_fcst_df.loc[row_idx, 'Humidity'] = float(data['fcstValue'])
        print('category:Humidity,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
    elif data['category']=='T3H':
        dangjin_fcst_df.loc[row_idx, 'Temperature'] = float(data['fcstValue'])
        print('category:Temperature,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
    elif data['category']=='SKY':
        dangjin_fcst_df.loc[row_idx, 'Cloud'] = float(data['fcstValue'])
        print('category:Cloud,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
    elif data['category']=='PTY':
        dangjin_fcst_df.loc[row_idx, 'PrecipitationForm'] = float(data['fcstValue'])
        print('category:PrecipitationForm,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])            
    elif data['category']=='POP':
        dangjin_fcst_df.loc[row_idx, 'PrecipitationProb'] = float(data['fcstValue'])
        print('category:PrecipitationProb,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])                       
    elif data['category']=='R06':
        dangjin_fcst_df.loc[row_idx, 'Precipitation'] = float(data['fcstValue'])
        print('category:Precipitation,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])                        
    elif data['category']=='S06':
        dangjin_fcst_df.loc[row_idx, 'Snow'] = float(data['fcstValue'])
        print('category:Snow,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])                                              
    elif data['category']=='VEC':
        dangjin_fcst_df.loc[row_idx, 'WindDirection'] = float(data['fcstValue'])
        print('category:WindDirection,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
    elif data['category']=='WSD':
        dangjin_fcst_df.loc[row_idx, 'WindSpeed'] = float(data['fcstValue'])
        print('category:WindSpeed,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'], '\n')
        row_idx+=3

00시를 제외한 23시 예보 데이터를 추출하여 덮어쓰기 합니다. <br>
00시 데이터는 위의 20시의 예보 데이터를 사용합니다. <br>

In [ ]:
dangjin_fcst_df

In [ ]:
dangjin_fcst_df = dangjin_fcst_df.interpolate()
dangjin_fcst = dangjin_fcst_df.iloc[:24]

In [ ]:
def make_locdate(x):
    Ymd, HMS = x.split(' ')
    Y, M, D = Ymd.split('-')
    return Y+M+D

dangjin_fcst['locdate'] = dangjin_fcst['Forecast_time'].apply(lambda x:make_locdate(x))

In [ ]:
site_info = pd.read_csv('./data/site_info.csv')
longi = site_info[(site_info['Id']=='당진태양광')]['Longitude'].values[0]
lati = site_info[(site_info['Id']=='당진태양광')]['Latitude'].values[0]
locdate = dangjin_fcst.loc[0,'locdate']

url = 'http://apis.data.go.kr/B090041/openapi/service/RiseSetInfoService/getLCRiseSetInfo?longitude='+str(longi)+'&latitude='+str(lati)+'&locdate='+str(locdate)+'&dnYn=Y&ServiceKey=API 키 값을 넣어주세요'
requestd = Request(url)
requestd.get_method = lambda: 'GET'
response_body = urlopen(requestd).read()
resp = xmltodict.parse(response_body)
resp_dic = json.loads(json.dumps(resp))
dangjin_fcst.loc[0, 'sunrise'] = resp_dic['response']['body']['items']['item']['sunrise']
dangjin_fcst.loc[0, 'sunset'] = resp_dic['response']['body']['items']['item']['sunset']
dangjin_fcst['sunrise'] = dangjin_fcst['sunrise'].fillna(method='ffill')
dangjin_fcst['sunset'] = dangjin_fcst['sunset'].fillna(method='ffill')
dangjin_fcst['sunrise'] = dangjin_fcst['sunrise'].astype(int)//100
dangjin_fcst['sunset'] = dangjin_fcst['sunset'].astype(int)//100

In [ ]:
dangjin_fcst = dangjin_fcst[['Forecast_time', 'Temperature', 'PrecipitationForm', 'PrecipitationProb', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud','Precipitation','Snow', 'sunrise', 'sunset']]
dangjin_fcst

## 울산 내일 기상 예보 받아오기(울산 남구 선암동 기준, x=102, y=83)

In [ ]:
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst?serviceKey=API 키 값을 넣어주세요&numOfRows=113&base_date='+str(today)+'&base_time=2000&nx=102&ny=83'

requestd = Request(url)
requestd.get_method = lambda: 'GET'
response_body = urlopen(requestd).read()
resp = xmltodict.parse(response_body)
response20 = json.loads(json.dumps(resp))

In [ ]:
ulsan_fcst_df = pd.DataFrame()

date = date.today() + timedelta(1)
ulsan_fcst_df['Forecast_time'] = [f'{date} {hour}:00' for hour in range(24)]
row_idx = 0 #20시 예보 데이터를 삽입하기 위한 초기위치 변수 입니다.

for i, data in enumerate(response20['response']['body']['items']['item']):
    if data['category']=='REH':
        ulsan_fcst_df.loc[row_idx, 'Humidity'] = float(data['fcstValue'])
        print('category:Humidity,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
    elif data['category']=='T3H':
        ulsan_fcst_df.loc[row_idx, 'Temperature'] = float(data['fcstValue'])
        print('category:Temperature,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
    elif data['category']=='SKY':
        ulsan_fcst_df.loc[row_idx, 'Cloud'] = float(data['fcstValue'])
        print('category:Cloud,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
    elif data['category']=='PTY':
        ulsan_fcst_df.loc[row_idx, 'PrecipitationForm'] = float(data['fcstValue'])
        print('category:PrecipitationForm,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])            
    elif data['category']=='POP':
        ulsan_fcst_df.loc[row_idx, 'PrecipitationProb'] = float(data['fcstValue'])
        print('category:PrecipitationProb,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])                       
    elif data['category']=='R06':
        ulsan_fcst_df.loc[row_idx, 'Precipitation'] = float(data['fcstValue'])
        print('category:Precipitation,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])                        
    elif data['category']=='S06':
        ulsan_fcst_df.loc[row_idx, 'Snow'] = float(data['fcstValue'])
        print('category:Snow,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])                                              
    elif data['category']=='VEC':
        ulsan_fcst_df.loc[row_idx, 'WindDirection'] = float(data['fcstValue'])
        print('category:WindDirection,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
    elif data['category']=='WSD':
        ulsan_fcst_df.loc[row_idx, 'WindSpeed'] = float(data['fcstValue'])
        print('category:WindSpeed,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'], '\n')
        row_idx+=3

3시간 간격으로 20시의 예보 데이터를 추출합니다.

In [ ]:
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst?serviceKey=API 키 값을 넣어주세요&numOfRows=113&base_date='+str(today)+'&base_time=2300&nx=102&ny=83'

requestd = Request(url)
requestd.get_method = lambda: 'GET'
response_body = urlopen(requestd).read()
resp = xmltodict.parse(response_body)
response23 = json.loads(json.dumps(resp))

In [ ]:
row_idx = 3 #00시를 제외한 23시 예보 데이터를 덮어쓰기하기 위한 초기위치 변수 입니다.

for i, data in enumerate(response23['response']['body']['items']['item']):
    if data['category']=='REH':
        ulsan_fcst_df.loc[row_idx, 'Humidity'] = float(data['fcstValue'])
        print('category:Humidity,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
    elif data['category']=='T3H':
        ulsan_fcst_df.loc[row_idx, 'Temperature'] = float(data['fcstValue'])
        print('category:Temperature,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
    elif data['category']=='SKY':
        ulsan_fcst_df.loc[row_idx, 'Cloud'] = float(data['fcstValue'])
        print('category:Cloud,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
    elif data['category']=='PTY':
        ulsan_fcst_df.loc[row_idx, 'PrecipitationForm'] = float(data['fcstValue'])
        print('category:PrecipitationForm,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])            
    elif data['category']=='POP':
        ulsan_fcst_df.loc[row_idx, 'PrecipitationProb'] = float(data['fcstValue'])
        print('category:PrecipitationProb,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])                       
    elif data['category']=='R06':
        ulsan_fcst_df.loc[row_idx, 'Precipitation'] = float(data['fcstValue'])
        print('category:Precipitation,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])                        
    elif data['category']=='S06':
        ulsan_fcst_df.loc[row_idx, 'Snow'] = float(data['fcstValue'])
        print('category:Snow,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])                                              
    elif data['category']=='VEC':
        ulsan_fcst_df.loc[row_idx, 'WindDirection'] = float(data['fcstValue'])
        print('category:WindDirection,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
    elif data['category']=='WSD':
        ulsan_fcst_df.loc[row_idx, 'WindSpeed'] = float(data['fcstValue'])
        print('category:WindSpeed,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'], '\n')
        row_idx+=3

00시를 제외한 23시 예보 데이터를 추출하여 덮어쓰기 합니다. <br>
00시 데이터는 위의 20시의 예보 데이터를 사용합니다. <br>

In [ ]:
ulsan_fcst_df = ulsan_fcst_df.interpolate()
ulsan_fcst = ulsan_fcst_df.iloc[:24]
ulsan_fcst['locdate'] = ulsan_fcst['Forecast_time'].apply(lambda x:make_locdate(x))

In [ ]:
longi = site_info[(site_info['Id']=='울산태양광')]['Longitude'].values[0]
lati = site_info[(site_info['Id']=='울산태양광')]['Latitude'].values[0]
locdate = ulsan_fcst.loc[0,'locdate']

url = 'http://apis.data.go.kr/B090041/openapi/service/RiseSetInfoService/getLCRiseSetInfo?longitude='+str(longi)+'&latitude='+str(lati)+'&locdate='+str(locdate)+'&dnYn=Y&ServiceKey=API 키 값을 넣어주세요'
requestd = Request(url)
requestd.get_method = lambda: 'GET'
response_body = urlopen(requestd).read()
resp = xmltodict.parse(response_body)
resp_dic = json.loads(json.dumps(resp))
ulsan_fcst.loc[0, 'sunrise'] = resp_dic['response']['body']['items']['item']['sunrise']
ulsan_fcst.loc[0, 'sunset'] = resp_dic['response']['body']['items']['item']['sunset']
ulsan_fcst['sunrise'] = ulsan_fcst['sunrise'].fillna(method='ffill')
ulsan_fcst['sunset'] = ulsan_fcst['sunset'].fillna(method='ffill')
ulsan_fcst['sunrise'] = ulsan_fcst['sunrise'].astype(int)//100
ulsan_fcst['sunset'] = ulsan_fcst['sunset'].astype(int)//100

In [ ]:
ulsan_fcst = ulsan_fcst[['Forecast_time', 'Temperature', 'PrecipitationForm', 'PrecipitationProb', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud','Precipitation','Snow', 'sunrise', 'sunset']]
ulsan_fcst

## preprocessing

In [ ]:
dangjin_sample = pd.read_csv('./new_data/dangjin_fcst_h2320.csv')
ulsan_sample = pd.read_csv('./new_data/ulsan_fcst_h2320.csv')
dangjin_all = pd.concat([dangjin_fcst,dangjin_sample], ignore_index=True)
ulsan_all = pd.concat([ulsan_fcst,ulsan_sample], ignore_index=True)

In [ ]:
def get_dircat(x):
    if x < 90:
        return 1
    elif (x >= 90)&(x < 180):
        return 2
    elif (x >= 180)&(x < 270):
        return 3
    else:
        return 4
    
def test_datast(test_df):
        
    test_df['Forecast_time2'] = pd.to_datetime(test_df['Forecast_time'])
    test_df['hour'] = test_df['Forecast_time2'].dt.hour
    test_df['year'] = test_df['Forecast_time2'].dt.year
    test_df['month'] = test_df['Forecast_time2'].dt.month
    test_df['day'] = test_df['Forecast_time2'].dt.day
    test_df['day_name'] = test_df['Forecast_time2'].dt.day_name()
    test_df['WindDirection'] = test_df['WindDirection'].apply(get_dircat)
    test_df['PP'] = test_df['PrecipitationProb']*test_df['PrecipitationForm']
    test_df['WW'] = test_df['WindSpeed']*test_df['WindDirection']
    test_df['HT'] = test_df['Humidity']*test_df['Temperature']
    
    test_df = test_df[['month', 'day', 'day_name', 'hour', 'Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud'
                      ,'PP','WW', 'HT']]
    test_df = pd.get_dummies(test_df, columns = ['hour', 'WindDirection', 'day_name'])                  
    
    return test_df

dangjin_test = test_datast(dangjin_all)
ulsan_test = test_datast(ulsan_all)
dangjin_test = dangjin_test.iloc[:24]
ulsan_test = ulsan_test.iloc[:24]

##### <span style="color:blue">* 위 코드의 하위 4줄에 대한 설명입니다. </span>


학습 데이터에는 존재했으나 추론 데이터가 존재하지 않는 값이 있을 때 <br>
get_dummies함수를 이용하여 one-hot encoding을 진행하면, <br>
학습 데이터와 추론 데이터의 특성 수가 달라진다. <br>
다른 입력 수로 인한 추론 실패를 방지하기 위하여 <br>
넓은 범위의 값이 존재하는 데이터 프레임에 concat을 하여 one-hot encoding을 진행한 뒤, <br>
추론에 필요한 행(ex. dangjin_test.iloc[:24])을 잘라내는 과정을 추가하였다. 

## predict

##### <span style="color:blue">* 개별 선별 작업을 거친 모델을 사용했습니다. </span>

DNN은 그 특성상 매번 모델의 성능이 달라집니다. <br>
public 평가 시 최고점수인 모델 세트 내의 개별 모델(4개)이 반드시 가장 좋은 모델임을 보장할 수 없기에 <br>
저희는 public 평가 점수 상위 5개의 모델 세트를 가지고 조합을 통해 <br> 
개별적으로 가장 뛰어난 모델을 선별하였습니다.  <br>
/best 디렉토리의 모델 세트는 그러한 선별을 거쳐  <br> 
private 평가에 사용하기 위한 최선의 모델을 선정한 결과입니다. <br>
따라서 저희는 /best의 모델 세트로 프라이빗 평가를 진행하였습니다.

In [ ]:
dangjin_floating_model = load_model('./best/dangjin_floating_model.h5')
dangjin_warehouse_model = load_model('./best/dangjin_warehouse_model.h5')
dangjin_model = load_model('./best/dangjin_model.h5')
ulsan_model = load_model('./best/ulsan_model.h5')

In [ ]:
dangjin_floating_pred = dangjin_floating_model.predict(dangjin_test)
dangjin_warehouse_pred = dangjin_warehouse_model.predict(dangjin_test)
dangjin_pred = dangjin_model.predict(dangjin_test)
ulsan_pred = ulsan_model.predict(ulsan_test)
plt.figure(figsize=(20,5))
plt.plot(dangjin_floating_pred, label = 'dangjin_floating_pred')
plt.plot(dangjin_warehouse_pred, label = 'dangjin_warehouse_pred')
plt.plot(dangjin_pred, label = 'dangjin_pred')
plt.plot(ulsan_pred, label = 'ulsan_pred')
plt.legend()
plt.show()

In [ ]:
if date.month == 6 :
    offset = 8
    day_idx = date.day - offset
    print('month :', date.month, 'index : ', day_idx)
elif date.month == 7 :
    offset = 22
    day_idx = offset + date.day
    print('month :', date.month, 'index : ', day_idx)
else :
    print('date error !!!!!')

##### <span style="color:blue">* 위 코드는 대회 기간동안 사용하기 위해 작성한 코드입니다. </span>

sample_submission 파일의 날짜를 맞추기 위해 <br>
올바른 인덱스를 얻는 코드입니다. <br>

In [ ]:
submission = pd.read_csv('./data/sample_submission.csv')
submission[648+(24*day_idx):672+(24*day_idx)]

In [ ]:
submission.iloc[648+(24*day_idx):672+(24*day_idx), 1] = dangjin_floating_pred
submission.iloc[648+(24*day_idx):672+(24*day_idx), 2] = dangjin_warehouse_pred
submission.iloc[648+(24*day_idx):672+(24*day_idx), 3] = dangjin_pred
submission.iloc[648+(24*day_idx):672+(24*day_idx), 4] = ulsan_pred
submission[648+(24*day_idx):672+(24*day_idx)]

In [ ]:
submission

In [ ]:
submission.to_csv(f'./submit/private/private_eval_best_h2320_{date}.csv', index=False)